In [1]:
# Init
# choropleth
from __future__ import division
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# https://www.census.gov/data/datasets/2019/econ/cbp/2019-cbp.html

df_raw = pd.read_csv('./raw_data/zip_business_profile_2019_raw.csv', na_values=".", encoding='latin-1')
# df_raw = pd.read_csv('./raw_data/zip_business_profile_2019_raw.csv', na_values=".")


In [3]:
# # Cleanup FIPS codes
# df_raw['fips'] = df_raw['fipstate'] * 1000 + df_raw['fipscty']

In [4]:
# Types of businesses
# https://www2.census.gov/programs-surveys/cbp/technical-documentation/reference/naics-descriptions/naics2017.txt
df = df_raw.query('naics == "------"')

In [11]:
df_raw

zip            name   naics  est  n<5 n5_9 n10_19 n20_49 n50_99  \
0          501  HOLTSVILLE, NY  ------    3    N    N      N      N      N   
1         1001      AGAWAM, MA  ------  473  217   89     69     60     23   
2         1001      AGAWAM, MA  23----   49   30    7      4      6      N   
3         1001      AGAWAM, MA  236///    5    N    N      N      N      N   
4         1001      AGAWAM, MA  2361//    4    N    N      N      N      N   
...        ...             ...     ...  ...  ...  ...    ...    ...    ...   
2881306  99999    UNCLASSIFIED  8139//   25   22    N      N      N      N   
2881307  99999    UNCLASSIFIED  81391/   12   11    N      N      N      N   
2881308  99999    UNCLASSIFIED  813910   12   11    N      N      N      N   
2881309  99999    UNCLASSIFIED  81392/   12   11    N      N      N      N   
2881310  99999    UNCLASSIFIED  813920   12   11    N      N      N      N   

        n100_249 n250_499 n500_999 n1000        city stabbr cty_name  
0              N        N        N     N  HOLTSVILLE     NY  SUFFOLK  
1             13        N        N     N      AGAWAM     MA  HAMPDEN  
2              N        N        N     N      AGAWAM     MA  HAMPDEN  
3              N        N        N     N      AGAWAM     MA  HAMPDEN  
4              N        N        N     N      AGAWAM     MA  HAMPDEN  
...          ...      ...      ...   ...         ...    ...      ...  
2881306        N        N        N     N                              
2881307        N        N        N     N                              
2881308        N        N        N     N                              
2881309        N        N        N     N                              
2881310        N        N        N     N                              

[2881311 rows x 16 columns]

In [5]:
# Renaming dataframe columns
# # https://www2.census.gov/programs-surveys/cbp/technical-documentation/records-layouts/2018_record_layouts/county-layout-2018.txt

df = pd.DataFrame()
df['zip'] = df_raw['zip']
# df['cbp_total_employees'] = df_raw['emp']
df['cbp_total_establishments'] = df_raw['est']

df['cbp_num_employees_5'] = df_raw['n<5']
df['cbp_num_employees_5_9']= df_raw['n5_9']
df['cbp_num_employees_10_19'] = df_raw['n10_19']
df['cbp_num_employees_20_49'] = df_raw['n20_49']
df['cbp_num_employees_50_99'] = df_raw['n50_99']
df['cbp_num_employees_100_249'] = df_raw['n100_249']
df['cbp_num_employees_250_499'] = df_raw['n250_499']
df['cbp_num_employees_500_999'] = df_raw['n500_999']
df['cbp_num_employees_1000'] = df_raw['n1000']
# df['cbp_num_employees_'] = df['N1000_1']
# df['cbp_num_employees_'] = df['N1000_2']
# df['cbp_num_employees_'] = df['N1000_3']
# df['cbp_num_employees_'] = df['N1000_4']


# N<5             N       Number of Establishments: Less than 5 Employee Size Class
# N5_9            N       Number of Establishments: 5-9 Employee Size Class
# N10_19          N       Number of Establishments: 10-19 Employee Size Class
# N20_49          N       Number of Establishments: 20-49 Employee Size Class
# N50_99          N       Number of Establishments: 50-99 Employee Size Class
# N100_249        N       Number of Establishments: 100-249 Employee Size Class
# N250_499        N       Number of Establishments: 250-499 Employee Size Class
# N500_999        N       Number of Establishments: 500-999 Employee Size Class
# N1000           N       Number of Establishments: 1,000 or More Employee Size Class
# N1000_1         N       Number of Establishments: Employment Size Class:
#                                 1,000-1,499 Employees
# N1000_2         N       Number of Establishments: Employment Size Class:
#                                 1,500-2,499 Employees
# N1000_3         N       Number of Establishments: Employment Size Class:
#                                 2,500-4,999 Employees
# N1000_4         N       Number of Establishments: Employment Size Class:
#                                 5,000 or More Employees

In [6]:
df

zip  cbp_total_establishments cbp_num_employees_5  \
0          501                         3                   N   
1         1001                       473                 217   
2         1001                        49                  30   
3         1001                         5                   N   
4         1001                         4                   N   
...        ...                       ...                 ...   
2881306  99999                        25                  22   
2881307  99999                        12                  11   
2881308  99999                        12                  11   
2881309  99999                        12                  11   
2881310  99999                        12                  11   

        cbp_num_employees_5_9 cbp_num_employees_10_19 cbp_num_employees_20_49  \
0                           N                       N                       N   
1                          89                      69                      60   
2                           7                       4                       6   
3                           N                       N                       N   
4                           N                       N                       N   
...                       ...                     ...                     ...   
2881306                     N                       N                       N   
2881307                     N                       N                       N   
2881308                     N                       N                       N   
2881309                     N                       N                       N   
2881310                     N                       N                       N   

        cbp_num_employees_50_99 cbp_num_employees_100_249  \
0                             N                         N   
1                            23                        13   
2                             N                         N   
3                             N                         N   
4                             N                         N   
...                         ...                       ...   
2881306                       N                         N   
2881307                       N                         N   
2881308                       N                         N   
2881309                       N                         N   
2881310                       N                         N   

        cbp_num_employees_250_499 cbp_num_employees_500_999  \
0                               N                         N   
1                               N                         N   
2                               N                         N   
3                               N                         N   
4                               N                         N   
...                           ...                       ...   
2881306                         N                         N   
2881307                         N                         N   
2881308                         N                         N   
2881309                         N                         N   
2881310                         N                         N   

        cbp_num_employees_1000  
0                            N  
1                            N  
2                            N  
3                            N  
4                            N  
...                        ...  
2881306                      N  
2881307                      N  
2881308                      N  
2881309                      N  
2881310                      N  

[2881311 rows x 11 columns]

In [7]:
# df[df.columns] = df[df.columns].replace(['N', 0], np.nan)
df[df.columns] = df[df.columns].replace(['N'], 0)


In [8]:
df

zip  cbp_total_establishments cbp_num_employees_5  \
0          501                         3                   0   
1         1001                       473                 217   
2         1001                        49                  30   
3         1001                         5                   0   
4         1001                         4                   0   
...        ...                       ...                 ...   
2881306  99999                        25                  22   
2881307  99999                        12                  11   
2881308  99999                        12                  11   
2881309  99999                        12                  11   
2881310  99999                        12                  11   

        cbp_num_employees_5_9 cbp_num_employees_10_19 cbp_num_employees_20_49  \
0                           0                       0                       0   
1                          89                      69                      60   
2                           7                       4                       6   
3                           0                       0                       0   
4                           0                       0                       0   
...                       ...                     ...                     ...   
2881306                     0                       0                       0   
2881307                     0                       0                       0   
2881308                     0                       0                       0   
2881309                     0                       0                       0   
2881310                     0                       0                       0   

        cbp_num_employees_50_99 cbp_num_employees_100_249  \
0                             0                         0   
1                            23                        13   
2                             0                         0   
3                             0                         0   
4                             0                         0   
...                         ...                       ...   
2881306                       0                         0   
2881307                       0                         0   
2881308                       0                         0   
2881309                       0                         0   
2881310                       0                         0   

        cbp_num_employees_250_499 cbp_num_employees_500_999  \
0                               0                         0   
1                               0                         0   
2                               0                         0   
3                               0                         0   
4                               0                         0   
...                           ...                       ...   
2881306                         0                         0   
2881307                         0                         0   
2881308                         0                         0   
2881309                         0                         0   
2881310                         0                         0   

        cbp_num_employees_1000  
0                            0  
1                            0  
2                            0  
3                            0  
4                            0  
...                        ...  
2881306                      0  
2881307                      0  
2881308                      0  
2881309                      0  
2881310                      0  

[2881311 rows x 11 columns]

In [9]:
# df = df[['fips', 'cbp_total_employees', 'cbp_total_establishments']]

In [10]:
df.to_csv('./processed_data/zip_business_profile_2019.csv')